In [2]:
# Imports ect
import numpy as np
import nnfs 
from nnfs.datasets import spiral_data

nnfs.init()

In [3]:
# Dense layer 
class Layer_Dense:
    
    # Layer Inintilization
    def __init__(self, n_inputs, n_neurons):
        # Initialize Weights and Biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    # Forward Pass
    def forward(self, inputs):
        # alculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases

In [4]:
# ReLU Acivation Function
class Activation_ReLU:
    
    # Forward Pass
    def forward(self, inputs):
        # Calculate the output value from the inputs
        self.output = np.maximum(0, inputs)

In [5]:
# Softmax Activation function
class Activtion_Softmax:
    
    # Forward Pass
    def forward(self, inputs):
        
        # Get un-normalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        # Normalize for each sample
        probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabilities

In [6]:
# Common Loss class
class Loss:
    
    # Calc data and regularisation losses
    # given model output and ground truth values
    def calculate(self, output, y):
        
        # Calc sample losses
        sample_losses = self.forward(output, y)
        
        # Calc mean loss
        data_loss = np.mean(sample_losses)
        
        return data_loss

In [7]:
# Cross-entropy Loss

class Loss_CategoricalCrossentropy(Loss):
    
    # Forward pass:
    def forward(self, y_pred, y_true):
        
        # No of smaples in a batch
        samples = len(y_pred)
        
        # Clip data to prevent division by 0
        # Clip both sides to avoid dragging mean to a particular value
        y_pred_clipped = np.clip(y_pred, 1e-7, 1- 1e-7)
        
        # Probabilities for target values -
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[
                range(samples),
                y_true
            ]
            
        # Mask values - only for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(
            y_pred_clipped*y_true,
            axis = 1
            )
        
        # Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods

In [8]:
# Create Data Set
X, y = spiral_data(points=100, classes=3)

In [9]:
# Create Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2,3)

# Create ReLU Activation Function (to be used with dense layer)
activation1 = Activation_ReLU()

# Create second dense layer (take outputs from previous layer as inputs) with 3 outputs
dense2 = Layer_Dense(3,3)

# Create Softmax Activation Function
activation2 = Activtion_Softmax()

# Create Loss Function
loss_function = Loss_CategoricalCrossentropy()

# Make a forward Pass of layer 1
dense1.forward(X)

# Make a forward pass of the 1st Activation Function
activation1.forward(dense1.output)

# Make a forward pass of the second layer
dense2.forward(activation1.output)

# Makes a forward pass of the 2nd Activation Function
activation2.forward(dense2.output)

# Print the first few results
print(activation2.output[:5])

# Perform a forward pass through activation function
# it takes the output of a second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

# Print loss value
print('Loss', loss)

[[0.33333334 0.33333334 0.33333334]
 [0.33333316 0.3333332  0.33333364]
 [0.33333287 0.3333329  0.33333418]
 [0.3333326  0.33333263 0.33333477]
 [0.33333233 0.3333324  0.33333528]]
Loss 1.0986104


In [11]:
# Calculate accuracy from the output of activation2 and targets
# calculate values along first axis
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y,axis=1)
accuracy= np.mean(predictions == y)

# Print accuracy
print('acc:', accuracy)

acc: 0.34
